In [1]:
pip install tweepy

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Dependencies and Setup
import tweepy as tw
from datetime import datetime,timedelta
import csv
import pandas as pd
from api_keys import consumer_key
from api_keys import consumer_secret
from api_keys import access_token
from api_keys import access_secret

In [16]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tw.API(auth)

In [17]:
number_of_tweets = 100

**Getting tweets**

In [50]:
# Define function to get the tweets

def get_tweets(search_words, date_since, language, number_of_items):
    
    tweets = tw.Cursor(api.search,
              q=search_words,
              lang=language,
              since=date_since
              ).items(number_of_items)
    
    tweet_list = []
    for tweet in tweets:
      tweet_list.append([tweet.created_at, tweet.text, tweet.id, tweet.user.screen_name,tweet.user.verified, tweet.retweet_count, 
                         tweet.favorite_count, tweet.user.followers_count,
                         tweet.user.friends_count, tweet.user.location])

# Create dataframe
    df = pd.DataFrame(tweet_list, columns=['Date', 'Tweet', 'Id', 'User_name', 'Verified', 'retweet_count', 'like_count',
                                           'followers_count','followees', 'location'])
    return df

In [51]:
# Call functions to create dataframes
df1 = get_tweets(['Paramount'], '2021-08-16', 'es', number_of_tweets)
df2 = get_tweets(['Disney Plus'], '2021-08-16', 'es', number_of_tweets)
df3 = get_tweets(['Amazon Prime Video'], '2021-08-16', 'es', number_of_tweets)
df4 = get_tweets(['HBO Max'], '2021-08-16', 'es', number_of_tweets)
df5 = get_tweets(['Start'], '2021-08-16', 'es', number_of_tweets)
df6 = get_tweets(['Apple TV'], '2021-08-16', 'es', number_of_tweets)
df7 = get_tweets(['Netflix'], '2021-08-16', 'es', number_of_tweets)

TweepError: Twitter error response: status code = 429

In [44]:
# Concatenate dataframe
frames = [df1, df2, df3, df4, df5, df6, df7]
result = pd.concat(frames)

In [45]:
# Dataframe
result

,Date,Tweet,Id,User_name,retweet_count,like_count,followers_count,followees,location
0,2021-08-21 17:14:28,Michael Bay seguirá siendo productor al igual ...,1429129788220977158,del_mutante,0,7,3877,685,Alcantarillas de New York
1,2021-08-21 17:11:46,"Una serie que les recomiendo: Yellowstone, con...",1429129111088287745,poterios,0,4,14827,1350,
2,2021-08-21 17:10:02,"#Musica\nCon un concierto inolvidable, en medi...",1429128675493130245,unpluggednewsmx,0,0,156,153,
3,2021-08-21 17:00:16,Thriller español Parot llega en exclusiva a Pa...,1429126217039556608,TheDailyTV,0,0,3403,3179,"Miami, EEUU"
4,2021-08-21 16:55:59,¿Quién tiene Paramount+? Es que quiero ver la ...,1429125140269436930,itsangelcandice,0,0,8148,3245,"𝓗𝓾𝓯𝓯𝓵𝓮𝓹𝓾𝓯𝓯. Éride, Ponque, Mai, Sere, Juan."
...,...,...,...,...,...,...,...,...,...
95,2021-08-21 17:12:23,RT @unchasqui: Qué fuerte la denuncia contra G...,1429129266059481088,JoeyL33082565,29,0,0,8,
96,2021-08-21 17:12:22,RT @PreviouslySerix: #NeverHaveIEver \n#YoNunc...,1429129259906478085,Stephany1228826,25,0,80,75,
97,2021-08-21 17:12:09,RT @diegoarroyogil: Me gustó mucho el document...,1429129205640478727,AnaMaDiLeo,5,0,5369,3056,"Caracas, Venezuela"
98,2021-08-21 17:11:58,"Hoy día de: beber, fumar, chuches y Netflix 🙂",1429129159557664772,MaleniRey15,0,0,19,60,


In [9]:
# Export to cvs sep without commas
result.to_csv('tweets.csv', sep='|', index=False )

**Previous steps**

In [ ]:
#tweet_list = []
    #for tweet in tweets:
      #tweet_list.append([tweet.created_at, tweet.text, tweet.id, tweet.retweet_count, 
                         #tweet.favorite_count, tweet.coordinates, tweet.user.followers_count,
                         #tweet.user.friends_count, tweet.user.location])

# Create dataframe
    #df = pd.DataFrame(tweet_list, columns=['Date', 'Tweet', 'Id', 'retweet_count', 'like_count',
                                           #'coordinates', 'followers_count','followees', 'location'])

In [ ]:
#search_words = ['Netflix OR Paramount OR Disney Plus OR HBO Max OR Start OR Apple TV OR Amazon Prime Video']
#search_words = ['Netflix']
#date_since = "2021-08-16"

In [ ]:
#tweets = tw.Cursor(api.search,
#              q=search_words,
#              lang="es",
#              since=date_since,
#              wait_on_rate_limit=True
#              ).items(100)

In [ ]:
#tweet_list = []
#with open('output2.csv', 'w') as f:
#  writer = csv.writer(f, delimiter='|')
#  for tweet in tweets:
#    writer.writerow([tweet.created_at, tweet.text])
#    tweet_list.append([tweet.created_at, tweet.text])

In [ ]:
#tweet_list = []
#for tweet in tweets:
#  tweet_list.append([tweet.created_at, tweet.text])

In [ ]:
#df = pd.DataFrame(tweet_list, columns=['Date', 'Tweet'])

In [ ]:
#df.head

In [ ]:
#search_words = ['Paramount']
#date_since = "2021-08-16"

In [ ]:
#tweets = tw.Cursor(api.search,
#              q=search_words,
#              lang="es",
#              since=date_since
#              ).items(1000)

In [ ]:
#tweet_list = []
#for tweet in tweets:
#  tweet_list.append([tweet.created_at, tweet.text])

In [ ]:
#df2 = pd.DataFrame(tweet_list, columns=['Date', 'Tweet'])

In [ ]:
#frames = [df, df2]
#result = pd.concat(frames)

In [ ]:
#result

,Date,Tweet
0,2021-08-21 04:33:06,☀️ Zenimation de Disney+ convierte escenas clá...
1,2021-08-21 04:25:01,Soy yo o la plataforma de disney plus está bie...
2,2021-08-21 04:17:14,"Amix, préstenme su cuenta de Disney plus y la ..."
3,2021-08-21 04:16:30,Una no puede ver una película online porque te...
4,2021-08-21 04:13:56,@jungkookanojo omgg yo ahora veo star vs las f...
...,...,...
995,2021-08-18 12:34:28,Que las marcas que hayan elegido para represen...
996,2021-08-18 12:34:25,"@JorgeRieraLopez No parece. Vendrá todo aquí, ..."
997,2021-08-18 12:33:51,"RT @enekoruizj: En 2022, tenemos nueva platafo..."
998,2021-08-18 12:33:37,El servicio de streaming SkyShowtime llegará a...
